In [1]:
!pip install nltk
!pip install gensim

     |████████████████████████████████| 1.5MB 1.5MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449907 sha256=d2a4aadede89b5667d9f93cbfa43df6188b93baf9f9dacf3cbaedd26bb4a53d2
  Stored in directory: /home/jovyan/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
     |████████████████████████████████| 24.2MB 4.6MB/s eta 0:00:01     |███████████████████▉            | 15.0MB 4.5MB/s eta 0:00:03     |█████████████████████▊          | 16.4MB 4.5MB/s eta 0:00:02     |█████████████████████████████▏  | 22.1MB 5.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 133kB 4.2MB/s eta 0:00:01
     |████████████████████████████████| 5.4MB 4.7MB/s eta 0:00:01     |██████▍                         | 1.1MB 4.1MB/s eta 0:00:02     |███████████████████████████████▌| 5.3MB 4.7MB/s e

In [1]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import re
import string
import pickle
from os import path
import gensim

In [2]:
data = pd.read_csv('~/work/data/ted-talks/transcripts.csv')
metadata = pd.read_csv('~/work/data/ted-talks/ted_main.csv')

In [3]:
data[:5]

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [4]:
metadata[:5]

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [5]:
data = data.join(metadata.set_index('url'), on='url')

In [6]:
data[data.transcript.isna()]

,transcript,url,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,views


In [7]:
data[:5]

,transcript,url,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,views
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,47227110
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,3200520
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,1636292
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,1697550
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,12005869


In [8]:
print("The attributes in the first dataset\n")
for col in data.columns:
    print(col)

The attributes in the first dataset

transcript
url
comments
description
duration
event
film_date
languages
main_speaker
name
num_speaker
published_date
ratings
related_talks
speaker_occupation
tags
title
views


In [10]:
tag2idx = {}
i=0
talktags = [re.sub('\'','',taglist)[1:-1].split(', ') for taglist in data['tags'].values]
for taglist in talktags:
    for tag in taglist:
        if tag not in tag2idx:
            tag2idx[tag] = i
            i += 1

In [12]:
data['transcript']

0       Good morning. How are you?(Laughter)It's been ...
1       Thank you so much, Chris. And it's truly a gre...
2       (Music: "The Sound of Silence," Simon & Garfun...
3       If you're here today — and I'm very happy that...
4       About 10 years ago, I took on the task to teac...
                              ...                        
2462    So, Ma was trying to explain something to me a...
2463    This is a picture of a sunset on Mars taken by...
2464    In my early days as a graduate student, I went...
2465    I took a cell phone and accidentally made myse...
2466    We humans are becoming an urban species, so ci...
Name: transcript, Length: 2467, dtype: object

In [13]:
data['transcript'] = data['transcript'].apply(lambda x: re.sub(r'\((.*?)\)', ' ', x))

In [14]:
def remove_punctuation(text):
    no_punc_text = [c  if c not in string.punctuation else ' ' for c in text]
    return no_punc_text

In [17]:
data['transcript'] = data['transcript'].apply(lambda x: nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(x.lower()))

In [18]:
def remove_stopwords(text):
    sw_text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
    return sw_text

In [20]:
nltk.download('stopwords')
data['transcript'] = data['transcript'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

def lemmatize(text):
    lem_text = [wordnet.morphy(w) if wordnet.morphy(w) else w for w in text]
    return lem_text

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [22]:
nltk.download('wordnet')
data['transcript'] = data['transcript'].apply(lambda x: lemmatize(x))

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
data['transcript'] = data['transcript'].apply(lambda x: ' '.join(x))

In [24]:
def sent_to_words(x):
        return gensim.utils.simple_preprocess(str(x), deacc=True)
    
data['transcript'] = data['transcript'].apply(lambda x: ' '.join(sent_to_words(x)))

In [25]:
data['transcript']

0       good morning great blow away whole thing fact ...
1       thank much chris truly great honor opportunity...
2       hello voice mail old friend call tech support ...
3       today happy hear sustainable development save ...
4       years ago take task teach global development s...
                              ...                        
2462    try explain something grandma grow pay attenti...
2463    picture sunset mars take nasa curiosity rover ...
2464    early days graduate student go snorkeling trip...
2465    take cell phone accidentally make famous talki...
2466    humans become urban species city natural habit...
Name: transcript, Length: 2467, dtype: object

In [26]:
print(data['transcript'].apply(lambda x: len(x.split(' '))).sort_values()[:20])

2301     1
1796     1
2000     1
146      1
908      1
1919     1
1173     1
1365     1
304      9
574     10
1105    41
266     42
1305    50
841     53
765     57
923     63
1890    64
760     64
85      67
381     78
Name: transcript, dtype: int64


In [29]:
data[data['transcript'].apply(lambda x: len(x.split(' '))) == 41].transcript.values

array(['today going talk unexpected discovery work solar technology industry small startup looking force environment pay attention pay attention crowd source quick video huh hang moment might take moment load skip skip video instead okay solar technology oh time okay thank much'],
      dtype=object)

In [30]:
data = data[data['transcript'].apply(lambda x: len(x.split(' '))) >= 41]

In [31]:
len(data)

2457

In [34]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer(min_df = 2)  
corpus_tfidf = tfidf.fit_transform(data['transcript'].values)

In [41]:
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

doc = data['transcript'][0]
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tfidf.transform([data['transcript'][0]]).tocoo())
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(tfidf.get_feature_names(),sorted_items,10)
 
# now print the results
print("\nAbstract:")
print(doc[:500],'...')
print("\nKeywords:")
for k in keywords:
    print(k,100*keywords[k]**2,'%')
print('\nTags')
print(data['tags'][0])


Abstract:
good morning great blow away whole thing fact leaving three theme running conference relevant want talk one extraordinary evidence human creativity presentation people variety range second put us place idea going happen terms future idea may play interest education actually find everybody interest education find interest dinner party say work education actually often dinner party frankly work education ask never ask back curiously strange say somebody know say say work education see blood run fa ...

Keywords:
education 9.7344 %
gillian 7.452900000000001 %
say 3.7636000000000003 %
think 2.56 %
stratford 2.4336 %
ballet 2.2201 %
shakespeare 1.8225000000000002 %
lynne 1.4641 %
dance 1.3456000000000001 %
educate 1.2769000000000001 %

Tags
['children', 'creativity', 'culture', 'dance', 'education', 'parenting', 'teaching']


In [42]:
with open(path.expanduser('~/work/test/tfidf/tfidf.pkl'), 'wb') as output:
    pickle.dump(tfidf, output, pickle.HIGHEST_PROTOCOL)

In [33]:
with open(path.expanduser('~/work/data/data.pkl'), 'wb') as output:
    pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)